In [1]:
import pymysql
import pymysql.cursors

In [2]:
conn = pymysql.connect(host='localhost', user = 'root', password='postech', db = 'jawiki', cursorclass=pymysql.cursors.DictCursor)
cursor = conn.cursor()

In [3]:
sql = 'SELECT *'
sql += 'FROM categorylinks A, page B '
sql += 'WHERE A.cl_from = B.page_id'
cursor.execute(sql)
category_link_data = cursor.fetchall()

In [4]:
len(category_link_data)

8224913

In [5]:
import pandas as pd
joined_data = pd.DataFrame(category_link_data, columns = ['cl_from', 'page_title', 'page_namespace', 'page_len','cl_type','cl_to'])
joined_data = joined_data[joined_data.page_title != joined_data.cl_to]

In [6]:
len(joined_data)

8177020

In [7]:
joined_data['page_title'] = joined_data['page_title'].str.decode("utf-8")
joined_data['cl_to'] = joined_data['cl_to'].str.decode("utf-8")

In [8]:
import numpy as np
joined_data['edge'] = list(zip(joined_data.page_title, joined_data.cl_to))
real_data = joined_data[(joined_data.page_namespace == 14)|(joined_data.page_namespace == 0)]

In [12]:
science_node = '科学'

In [13]:
tech_node = '技術'

In [14]:
delete_list_science = list(real_data[real_data.page_title==science_node]['cl_to'])

In [15]:
delete_list_tech = list(real_data[real_data.page_title==tech_node]['cl_to'])

In [16]:
delete_list_science

['GND識別子が指定されている記事',
 'ISBNマジックリンクを使用しているページ',
 'LCCN識別子が指定されている記事',
 '和製漢語',
 '哲学の和製漢語',
 '学問',
 '技術',
 '知識']

In [17]:
delete_list_tech

['出典を必要とする記事/2013年8月', '哲学の和製漢語', '大言壮語的な記述になっている項目', '工学関連のスタブ', '主要カテゴリ']

In [28]:
real_data = real_data[real_data.page_title!=science_node]

In [29]:
real_data = real_data[real_data.page_title!=tech_node]

In [31]:
edge_list = list(real_data.edge)

In [32]:
root_node = 'Science_and_technology_ja'

In [33]:
edge_list.append((science_node,root_node))

In [34]:
edge_list.append((tech_node,root_node))

In [35]:
len(edge_list)

7185564

In [36]:
def size_of_networks(graph):
    return {'number_of_nodes': len(graph.nodes()), 'number_of_links': len(graph.edges()), 'ratio': len(graph.edges())/len(graph.nodes())}

In [46]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(edge_list)

In [47]:
size_of_networks(G)

{'number_of_links': 7143599,
 'number_of_nodes': 1343951,
 'ratio': 5.315371616971154}

In [48]:
node_set = list(nx.ancestors(G, root_node)) + [root_node]

In [54]:
len(nx.ancestors(G, root_node))

1332458

In [55]:
len(nx.descendants(G, root_node))

0

In [56]:
subgraph = nx.subgraph(G, node_set)

In [57]:
nx.write_gml(subgraph, 'network_data/jawiki.gml')
pd.Series(subgraph.nodes()).to_pickle('network_data/jawiki.pkl')

In [58]:
size_of_networks(subgraph)

{'number_of_links': 7129762,
 'number_of_nodes': 1332459,
 'ratio': 5.350830306973798}